https://github.com/tensorflow/serving/issues/310

In [1]:
import sys
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
import os

Using TensorFlow backend.


In [2]:
# Input size of the network, the entry text must have the same length
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()

In [3]:
from keras.layers import InputLayer
def create_gru_model( num_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model

In [4]:
import keras.backend as K
K.set_learning_phase(0)

number_chars=len(chars_to_indices)
model=create_gru_model(number_chars)
model.load_weights('../model_weights/best_beiras_gru_textdata_weights.hdf5')


https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

In [5]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

In [6]:
export_path = "../export-tf/1"
import shutil
import os


if os.path.isdir(export_path):
    shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'sequence': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'predict': signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../export-tf/1/saved_model.pb'


In [7]:
print(signature)
signature_def_map={'predict': signature}
print(signature_def_map)
    

inputs {
  key: "inputs"
  value {
    name: "gru_1_input:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 100
      }
      dim {
        size: 55
      }
    }
  }
}
outputs {
  key: "outputs"
  value {
    name: "dense_1/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 55
      }
    }
  }
}
method_name: "tensorflow/serving/predict"

{'predict': inputs {
  key: "inputs"
  value {
    name: "gru_1_input:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 100
      }
      dim {
        size: 55
      }
    }
  }
}
outputs {
  key: "outputs"
  value {
    name: "dense_1/Softmax:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 55
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}


Prerequisitos installas tennsoflow servein
echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list

curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

sudo apt-get update && sudo apt-get install tensorflow-model-server


Dentro do directorio ..export-tf creemos dir 1 e movemos todo ahi, esta seria version 1.


tensorflow_model_server --port=9000 --model_name=mnist --model_base_path=/tmp/mnist_model/



In [7]:
print(model.input)

Tensor("gru_1_input:0", shape=(?, 100, 55), dtype=float32)


In [8]:
print(model.output)

Tensor("dense_1/Softmax:0", shape=(?, 55), dtype=float32)


https://gist.github.com/avloss/01e43d208fbdb2c5b4f9b50e71617cc8

tensorflow_model_server --port=9000 --model_name=default --model_base_path=/home/aind2/beiras-rnn/export-tf


In [1]:
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
input_clean=clean_text(input_init.lower())
if len(input_clean) < window_size:
    print("Sentence must have ", window_size, len(input_sentence))
else:
    input_clean = input_clean[:window_size]
    x_test = np.zeros((1,window_size, number_chars))
    for t, char in enumerate(input_clean):
        x_test[0, t, chars_to_indices[char]] = 1.
    
    


In [2]:
x_test

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]])

In [8]:
x_test.dtype

dtype('float64')

O file pb describe o modelo
Si cambio a builder.save(astext=True) xera o pb en modo texto

https://medium.com/towards-data-science/how-to-deploy-machine-learning-models-with-tensorflow-part-2-containerize-it-db0ad7ca35a7

git clone --recurse-submodules https://github.com/tensorflow/serving.git

cd <tensorflow serving source folder>
# 2
mv ./tensorflow ./tensorflow_
mv ./tensorflow_/tensorflow .
# 3
python -m grpc.tools.protoc ./tensorflow_serving/apis/*.proto --python_out=<path to GAN project> --grpc_python_out=<path to GAN project> --proto_path=.
# 4
mv ./tensorflow ./tensorflow_
mv ./tensorflow_ ./tensorflow

In [10]:
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import tensorflow as tf
import grpc

channel = grpc.insecure_channel("localhost:" + str(9000))
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
request = predict_pb2.PredictRequest() 
request.model_spec.name = 'default' 
request.model_spec.signature_name = 'predict' 

request.inputs['inputs'].CopyFrom( 
        tf.contrib.util.make_tensor_proto(
            x_test,dtype='float32'))
result=stub.Predict(request)


outputs {
  key: "outputs"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 55
      }
    }
    float_val: 0.0024777590297162533
    float_val: 5.75096123067026e-13
    float_val: 0.0002520043053664267
    float_val: 8.448802191196592e-08
    float_val: 7.325351703002525e-07
    float_val: 2.476176177879097e-06
    float_val: 3.2858672511792975e-06
    float_val: 1.2969632734893821e-05
    float_val: 9.335383879260917e-07
    float_val: 1.114238898480835e-06
    float_val: 2.3529828752089088e-07
    float_val: 3.3810536024247995e-06
    float_val: 3.2726774179536733e-07
    float_val: 1.7758783315002802e-06
    float_val: 1.1227526556467637e-05
    float_val: 0.012260417453944683
    float_val: 2.9183180231484585e-05
    float_val: 0.0027405295986682177
    float_val: 3.139560067211278e-05
    float_val: 0.0013600774109363556
    float_val: 2.39297008874928e-07
    float_val: 2.9464549697877374e-06
    float_val: 1.07620107

In [24]:
test_predict=np.array(result.outputs['outputs'].float_val)
r = np.argmax(test_predict)  # predict class of each test input
d = indices_to_chars[r]
d

'm'

In [28]:
def predict_one(text_predict,stub):
    x_test = np.zeros((1,window_size, number_chars))
    for t, char in enumerate(input_clean):
        x_test[0, t, chars_to_indices[char]] = 1.
    request = predict_pb2.PredictRequest() 
    request.model_spec.name = 'default' 
    request.model_spec.signature_name = 'predict' 
    request.inputs['inputs'].CopyFrom( 
        tf.contrib.util.make_tensor_proto(
            x_test,dtype='float32'))
    result=stub.Predict(request)
    test_predict=np.array(result.outputs['outputs'].float_val)
    r = np.argmax(test_predict)  # predict class of each test input
    return (indices_to_chars[r])

In [29]:
input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
input_clean=clean_text(input_init.lower())
channel = grpc.insecure_channel("localhost:" + str(9000))
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
d=predict_one(input_clean,stub)
print(d)

m


In [39]:
def predict_window(text_predict,number_predict):
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    input_clean=clean_text(text_predict.lower())
    channel = grpc.insecure_channel("localhost:" + str(9000))
    stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
    for i in range(number_predict):
        print(i)
        print(input_clean)
        print(input_clean[i:])
        d=predict_one(input_clean[i:],stub)
        input_clean+=d
    return input_clean
        

In [40]:
text=predict_window(input_init,window_size)

0
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu
1
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcum
e moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcum
2
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcumm
 moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcumm
3
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcummm
moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcummm
4
se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcummmm
oito cando dixen eu que as suas políticas agresoras do común cidadán ma

In [36]:
text

'se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcummmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm'